# BERT Question Answering Bot



In [ ]:
!pip install pytorch-pretrained-bert pytorch-nlp pytorch_transformers
# !pip install transformers

In [ ]:
import tensorflow as tf
import random
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_transformers import BertTokenizer, BertConfig, BertModel, AdamW, BertForQuestionAnswering
from tqdm import tqdm, trange
import pandas as pd
import io
import os
import numpy as np
import matplotlib.pyplot as plt
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification, BertTokenizer, BertConfig
from tqdm import tqdm, trange
import textwrap
% matplotlib inline



In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

Found GPU at: /device:GPU:0


'Tesla V100-SXM2-16GB'

## Import Data

In [ ]:
from google.colab import drive
drive.mount('/drive',force_remount=True)

Mounted at /drive


In [ ]:
!ls /drive/My\ Drive/qa_bot

cache	     models.py	  transformer		   utils_squad.py
data	     predictions  utils_preprocess.py	   visualizations.py
evaluate.py  __pycache__  utils_squad_evaluate.py


In [ ]:
import sys
sys.path.append('/drive/My Drive/qa_bot')

In [ ]:
from utils_squad import (read_squad_examples, convert_examples_to_features,
                         RawResult, write_predictions,
                         RawResultExtended, write_predictions_extended)
from utils_squad_evaluate import EVAL_OPTS, main as evaluate_on_squad, plot_pr_curve

In [ ]:
input_file = '/drive/My Drive/qa_bot/data/train-v2.0.json'
examples_train = read_squad_examples(input_file=input_file,
                                is_training=True,
                                version_2_with_negative=True)

In [ ]:
train_data = pd.DataFrame.from_records([vars(ex) for ex in examples_train])
train_data.head()

,qas_id,question_text,doc_tokens,orig_answer_text,start_position,end_position,is_impossible
0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,"[Beyoncé, Giselle, Knowles-Carter, (/biːˈjɒnse...",in the late 1990s,39,42,False
1,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,"[Beyoncé, Giselle, Knowles-Carter, (/biːˈjɒnse...",singing and dancing,28,30,False
2,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,"[Beyoncé, Giselle, Knowles-Carter, (/biːˈjɒnse...",2003,82,82,False
3,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,"[Beyoncé, Giselle, Knowles-Carter, (/biːˈjɒnse...","Houston, Texas",22,23,False
4,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,"[Beyoncé, Giselle, Knowles-Carter, (/biːˈjɒnse...",late 1990s,41,42,False


# Preprocessing

In [ ]:
# Specify tokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
doc_stride = 128
max_seq_length = 256
max_query_length = 64
batch_size = 32

In [ ]:
if not os.path.exists('/drive/My Drive/qa_bot/cache'):
    os.makedirs('/drive/My Drive/qa_bot/cache')
cached_features_file = '/drive/My Drive/qa_bot/cache/cache_train'

In [ ]:
from utils_preprocess import create_features
features_train = create_features(cached_features_file = cached_features_file,
                           examples=examples_train,
                           tokenizer=tokenizer,
                           max_seq_length=max_seq_length,
                           doc_stride=doc_stride,
                           max_query_length=max_query_length,
                           is_training=True)

In [ ]:
# Build training data
from utils_preprocess import generate_bert_loader_train
train_dataloader, len_train = generate_bert_loader_train(features_train, batch_size=batch_size, drop_last=True)

## Fine-Tune Model

In [ ]:
def set_seed(seed=0):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [ ]:
# Specify architecture for output layer and whether to freeze bert parameters
from models import bert_mod
bert = bert_mod('bert-base-uncased')
# bert.freeze_param()

# model_name = "bert_linear"
# model = bert.bert_linear()

# model_name = "bert_linear_linear"
# model = bert.bert_linear_linear()

model_name = "bert_linear_act_drop"
model = bert.bert_linear_act_drop()

# model_name = "bert_linear_more_hidden"
# model = bert.bert_linear_more_hidden()

# model_name = "bert_linear_linear_linear"
# model = bert.bert_linear_linear_linear()

# model_name = "bert_linear_full"
# model = bert.bert_linear_full()

# model_name = "bert_linear_no_act"
# model = bert.bert_linear_no_act()

# model_name = "bert_conv_linear"
# model = bert.bert_conv_linear()

# print out model architecture
model.cuda()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [ ]:
global_step = 0
train_loss_set = []
tr_loss = 0.0
learning_rate = 5e-5
adam_epsilon=1e-8
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.1},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.1}
    ]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=adam_epsilon,betas=(0.9,0.999))

In [ ]:
num_train_epochs = 1

print("***** Running training *****")
print("  Num examples = %d" % len_train)
print("  Num Epochs = %d" % num_train_epochs)
print("  Batch size = %d" % batch_size)
print("  Total optimization steps = %d" % (len(train_dataloader) // num_train_epochs))

model.zero_grad()
train_iterator = trange(num_train_epochs, desc="Epoch")
set_seed()

for _ in train_iterator:
    epoch_iterator = tqdm(train_dataloader,desc="iterations",disable=False,position=0, leave=True)
    for step, batch in enumerate(epoch_iterator):
      if step < int(global_step) + 1:
        continue

      model.train()
      batch = tuple(t.to(device) for t in batch)
      inputs = {'input_ids':       batch[0],
                'attention_mask':  batch[1], 
                'token_type_ids':  batch[2],  
                'start_positions': batch[3], 
                'end_positions':   batch[4]}

      outputs = model(**inputs)
      loss = outputs[0]
      train_loss_set.append(loss)
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
      tr_loss += loss.item()
      optimizer.step()
      model.zero_grad()
      global_step += 1


In [ ]:
# Plot training loss
# model_name = "bert_linear_more_hidden (Unfrozen)"
plt.figure(figsize=(15,8))
plt.title("Training loss: "+ model_name)
plt.xlabel("Batch")
plt.ylabel("Loss")
plt.plot(train_loss_set)
plt.show()

## Evaluate Result

In [ ]:
input_file = '/drive/My Drive/qa_bot/data/dev-v2.0.json'
examples_val = read_squad_examples(input_file=input_file,
                                is_training=False,
                                version_2_with_negative=True)
doc_stride = 128
max_seq_length = 256
max_query_length = 64
batch_size = 32
cached_features_file = '/drive/My Drive/qa_bot/cache/cache_validation'

# Cache features
features_val = create_features(cached_features_file=cached_features_file,
                                    examples=examples_val,
                                    tokenizer=tokenizer,
                                    max_seq_length=max_seq_length,
                                    doc_stride=doc_stride,
                                    max_query_length=max_query_length,
                                    is_training=False)

In [ ]:
from utils_preprocess import generate_bert_loader_validation
validation_dataloader, len_val = generate_bert_loader_validation(features_val, batch_size=batch_size, drop_last=True)

In [ ]:
print(model_name)
path_val = '/drive/My Drive/qa_bot/data/dev-v2.0.json'
from evaluate import evaluate
results = evaluate(model, 
                   validation_dataloader, 
                   features_val,
                   examples_val,
                   tokenizer, 
                   path_val, 
                   len_val, 
                   batch_size,
                   device)



In [ ]:
import json
results_json = []
for k in enumerate(results.keys()):
  result_dict = {k[1] : results[k[1]]}
  results_json.append(result_dict)
print(results_json)
with open('results.json', 'w') as f:
  json.dump(results_json, f)